In [28]:
!pip install -U spacy
import spacy
nlp = spacy.load("en_core_web_sm")
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Baseline_MUSS_corrupt/muss
!pip install -e . 

import random
import re
import math
import pickle
from itertools import product
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import os
%cd '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/'
# !pip install imohash

from muss.utils.helpers import *


train_df = pd.read_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/train_df_0911")
valid_df = pd.read_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/valid_df_0911")
test_df = pd.read_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/test_df_0911")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 25.1 MB/s 
     |████████████████████████████████| 458 kB 38.6 MB/s 
     |████████████████████████████████| 821 kB 29.1 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.5
    Uninstalling thinc-7.4.5:
      Successfully uninstalled thinc-7.4.5
  Attempting uninstall: spacy
    Found existing installation: spacy 2.3.7
    Uninstalling spacy-2.3.7:
      Successfully uninstalled spacy-2.3.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Baseline_MUSS_corrupt/muss
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/Baseline_MUSS_corrupt/muss
  Cloning https://github.com/feralvam/easse.git to /tmp/pip-install-gaci0ztm/easse_f908015a5aa64bc2b5e3ad7fdac2471d
  Running command git clone -q https://github.com/feralvam/easse.git /tmp/pip-install-gaci0ztm/easse_f908015a5aa64bc2b5e3ad7fdac2471d
  Cloning https://github.com/kpu/kenlm.git to /tmp/pip-install-gaci0ztm/kenlm_05e337e800c34608bcc58443e642464c
  Running command git clone -q https://github.com/kpu/kenlm.git /tmp/pip-install-gaci0ztm/kenlm_05e337e800c34608bcc58443e642464c
  Using cached spacy-2.3.7-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.4 MB)
  Cloning https://github.com/facebookresearch/text-simplification-eval

/content/drive/MyDrive/Baseline_MUSS_corrupt/muss


In [29]:
from muss.mining.training import get_bart_kwargs, get_score_rows
from muss.resources.prepare import prepare_wikilarge_detokenized, prepare_asset
from muss.resources.datasets import create_smaller_dataset

from muss.fairseq.main import fairseq_train_and_evaluate_with_parametrization
from muss.mining.training import get_score_rows #get_bart_kwargs

#########################
import re
import shutil
import os 

from cachier import cachier
from easse.cli import evaluate_system_output
from easse.utils.constants import TEST_SETS_PATHS
import torch
from tqdm import tqdm

from muss.resources.paths import get_data_filepath, MODELS_DIR, get_dataset_dir
from muss.utils.helpers import add_dicts, args_str_to_dict
from muss.utils.resources import download_and_extract
from muss.preprocessors import GPT2BPEPreprocessor
from muss.preprocessing import apply_line_function_to_file
from muss.fairseq.main import get_language_from_dataset
from muss.text import truncate

from muss.simplify import simplify_sentences
from muss.utils.helpers import *

from muss.simplifiers import get_fairseq_simplifier, get_preprocessed_simplifier
from muss.preprocessors import get_preprocessors, get_preprocessor_by_name
# from muss.utils.eval_utils import *

In [30]:
from muss.utils.eval_utils import *

In [31]:
# !git clone https://github.com/feralvam/easse.git
!cd easse
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/Baseline_MUSS_corrupt/muss
  Cloning https://github.com/feralvam/easse.git to /tmp/pip-install-n6z1mnrl/easse_dccd55eb2a2e4385a226059e6c7288d2
  Running command git clone -q https://github.com/feralvam/easse.git /tmp/pip-install-n6z1mnrl/easse_dccd55eb2a2e4385a226059e6c7288d2
  Cloning https://github.com/kpu/kenlm.git to /tmp/pip-install-n6z1mnrl/kenlm_9133bd1e677946b080d7748b72d5b911
  Running command git clone -q https://github.com/kpu/kenlm.git /tmp/pip-install-n6z1mnrl/kenlm_9133bd1e677946b080d7748b72d5b911
  Cloning https://github.com/facebookresearch/text-simplification-evaluation.git (to revision main) to /tmp/pip-install-n6z1mnrl/tseval_366f6321158646a7891cf943ba7660d4
  Running command git clone -q https://github.com/facebookresearch/text-simplification-evaluation.git /tmp/pip-install-n6z1mnrl/tseval_366f6321158646a7891cf943ba7660d4
  Attem

In [32]:
muss_path = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/'
# while True: pass

### Simplify asset.valid

In [33]:
### Compute hardword word retention rate 
def find_occurence(NE,simple_sentence):
  NE_simp = [i.lower() for i in NE['NE_simple']]
  NE_diff = [i.lower() for i in NE['NE_difficult']]
  simple_sentence = simple_sentence.lower()
  intersect_words = []

  count_simple = 0
  count_difficult = 0
  # print(NE_diff)
  # print(NE)
  if NE_simp[0] != '' or NE_diff:
    # print(NE_simp[0] == '')
  # print(sentence,test_complex_NER_spacy[index])
    if NE_simp[0] != '':
      for text in NE_simp:
        if text.lower() in simple_sentence:
          intersect_words.append(text)
          count_simple +=1
    
    if NE_diff:
      for text in NE_diff:
        if text.lower() in simple_sentence:
          intersect_words.append(text)
          count_difficult +=1
          # print('diff words found in sen')

    return count_simple, count_difficult
  else:
    # print('empty NE')
    return 0,0
    
def word_retention_rate(comp_sentence,simp_sentence):
  total_simp = 0
  total_diff = 0
  counter_simp = 0
  counter_diff = 0

  for i,sentence in enumerate(simp_sentence):
    NE_temp1 = NE_extraction(comp_sentence[i])[0]

    count_simp, count_diff = find_occurence(NE_temp1,sentence)

    total_simp += len(NE_temp1['NE_simple'])
    total_diff += len(NE_temp1['NE_difficult'])

    counter_simp += count_simp
    counter_diff += count_diff
    # print(count_simp, count_diff)
  if total_diff != 0:
    print('simple_NE retention rate',counter_simp/total_simp)
    print('difficult_NE retention rate',counter_diff/total_diff)
    print('total_NE retention rate', (counter_simp+counter_diff)/(total_simp+total_diff))
    return counter_simp/total_simp,counter_diff/total_diff, (counter_simp+counter_diff)/(total_simp+total_diff)
  else:
    print('simple_NE retention rate',counter_simp/total_simp)
    return counter_simp/total_simp

def generate_output(chosen_id_list,data_dir_list,data_list,model_dir_dict,sample=False,add_to_dir_name=None,muss_output=False,NE_output=False,CERF_output=False,NE_CERF_output=False,hypothesis_num=None,beam=None,sampling=False):

  # store created dir
  created_dir = []
  output_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/'
  for index in chosen_id_list:
    
    # get model unique local id
    exp_dir = model_dir_dict[index]['exp_dir']
    print('| model_name is',model_dir_dict[index]['model_name'])
    print('| exp_dir is',exp_dir)

    model_name = 'model_'+ str(index) + '_'+ exp_dir.split('/')[-2]

    os.makedirs(output_dir+model_name, exist_ok=True)
    print('| output_dir',output_dir)
    language = 'en'

    # get recommended_preprocessors_kwargs
    if 'recommended_preprocessors_kwargs' in model_dir_dict[index].keys() :
      preprocessors_kwargs = model_dir_dict[index]['recommended_preprocessors_kwargs']
      print('| use provided kwargs')
    else:
      # use muss_mined_wiki preprocessors
      preprocessors_kwargs = {
            'LengthRatioPreprocessor': {'target_ratio': 0.9, 'use_short_name': False},
            'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.65, 'use_short_name': False},
            'WordRankRatioPreprocessor': {'target_ratio': 0.75, 'language': language, 'use_short_name': False},
            'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 0.4, 'language': language, 'use_short_name': False},
        }
      preprocessors_kwargs['GPT2BPEPreprocessor'] = {}

    print('| use preprocessors_kwargs',preprocessors_kwargs)
    preprocessors = get_preprocessors(preprocessors_kwargs)

    if hypothesis_num == None:
      hypothesis_num = 1
    if beam == None:
      beam=5

    generate_kwargs = {'hypothesis_num':hypothesis_num,
                       'beam':beam,
                       'sampling':sampling}

    simplifier = get_fairseq_simplifier(exp_dir, **generate_kwargs)
    simplifier = get_preprocessed_simplifier(simplifier, preprocessors=preprocessors)

    #########################################
    # simplify sample sentences 

    ############################## I need to change the directory 
    if sample:
      if muss_output:
        complex_file_dir = '/content/drive/MyDrive/muss/scripts/contract_no_token.en'
      elif NE_output:
        complex_file_dir = '/content/drive/MyDrive/muss/scripts/contract_NE_token.en'
      elif CERF_output:
        complex_file_dir = '/content/drive/MyDrive/muss/scripts/contract_ABCD_token.en'
      elif NE_CERF_output:
        complex_file_dir = '/content/drive/MyDrive/muss/scripts/contract_NE_ABCD_token.en'

      pred_path = simplifier(complex_file_dir)

      for i in range(len(read_lines(complex_file_dir))):
        print('original:\n',read_lines(complex_file_dir)[i])
        print('simplified:\n',read_lines(pred_path)[i])
        print('----------------------------------------------------')

      return read_lines(complex_file_dir),read_lines(pred_path)
    ###########################################

    # create dir for output
    subfolders = os.listdir(output_dir + model_name)
    num_subfolder = len(subfolders)
    new_folder_dir = output_dir + model_name + '/' +str(num_subfolder).zfill(2) +'/'
    os.mkdir(new_folder_dir)
    created_dir.append(new_folder_dir)

     # simplify asset dataset
    for index, file_dir in enumerate(data_dir_list):
      
      info_file_name = new_folder_dir + 'info.txt'
      with open(info_file_name, 'w') as f:
        f.write("generate_kwargs %s\n" % generate_kwargs)
      f.close()

      filename = new_folder_dir + data_list[index]
      print('filename;', filename)
      pred_path = simplifier(file_dir)

      with open(filename, 'w') as f:
          for item in read_lines(pred_path):
              f.write("%s\n" % item)

  print('created_dir',created_dir)
  return created_dir

In [1]:
MODEL_DIR = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/'
Temp_data_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/'
Temp_data_name = 'process_sentence1'
model_dir_dict = {}

test_or_valid = 'test'

def add_item_to_dict(**kwargs):

  id = len(model_dir_dict)
  model_dir_dict[id]={}

  for key, value in kwargs.items():
    model_dir_dict[id][key]=value
  print('added:',model_dir_dict[id])

add_item_to_dict(model_id=len(model_dir_dict),exp_dir= MODEL_DIR+'local_1660919962534/',model_name='WIKI_Simplify')
test_data_dir_list,test_data_list = ['/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset_ne/'+test_or_valid+'.complex'],['valid.diff.complex.pred']
model_dir_dictx = model_dir_dict
model_dir_dictx[0]

added: {'model_id': 0, 'exp_dir': '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/local_1660919962534/', 'model_name': 'WIKI_Simplify'}


{'model_id': 0,
 'exp_dir': '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/local_1660919962534/',
 'model_name': 'WIKI_Simplify'}

In [2]:
token_ori_1 = generate_output([0],test_data_dir_list,test_data_list, model_dir_dictx)

NameError: ignored

In [68]:
# comp_sentence, simp_sentence = show_simplify(test_data_dir_list[0],token_ori_1[0]+test_data_list[0])

test_pred_dir = token_ori_1[0]+test_data_list[0]
# read_lines(test_pred_dir)
asset_data_path_ne = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset_ne/'
asset_data_path = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/'


ref_data = []
for i in range(0,10):
  ref_data.append(asset_data_path+test_or_valid+'.simple.'+str(i))
ref_data

orig_data = asset_data_path +test_or_valid+'.complex'
print('ref_data')
# read_lines(ref_data[0])
print(ref_data)

ref_data
['/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/test.simple.0', '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/test.simple.1', '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/test.simple.2', '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/test.simple.3', '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/test.simple.4', '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/test.simple.5', '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/test.simple.6', '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/test.simple.7', '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/test.simple.8', '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/test.simple.9']


In [73]:
!easse evaluate -t turkcorpus_test -m 'bleu,sari,fkgl' -q < easse

Fatal Python error: _PySys_BeginInit: <stdin> is a directory, cannot continue

Current thread 0x00007f34faaa4780 (most recent call first):


In [75]:
!easse evaluate -refs_sents_paths ref_data -orig_sents_path orig_data -sys_sents_path test_pred_dir -t custom -m 'bleu,sari,fkgl' -q < '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/'


Fatal Python error: _PySys_BeginInit: <stdin> is a directory, cannot continue

Current thread 0x00007f52d8f6c780 (most recent call first):


In [78]:
os.popen("easse evaluate -t turkcorpus_test -m 'bleu,sari,fkgl' -q < %s" %'/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/preprocess').read()

''

In [46]:
from easse.sari import corpus_sari
from easse.bleu import corpus_bleu
from easse.sari import corpus_sari

In [83]:

# corpus_bleu(sys_sents=['haha is a good joke alright'],refs_sents=[['haha is a good joke']])
# !easse evaluate -t turkcorpus_test -m 'bleu,sari,fkgl' -q < '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/preprocess.low'
!easse -h Usage

Usage: easse [OPTIONS] COMMAND [ARGS]...

Options:
  --version   Show the version and exit.
  -h, --help  Show this message and exit.

Commands:
  evaluate
  report


In [ ]:
def easse_my_model_eval(output_id,generate_id,dataset,phase,output_file_name):

  pred_path = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/'+output_id+'/'+generate_id+'/'+output_file_name

  print('pred_path',pred_path)
  if phase=='valid':
    return os.popen("easse evaluate -t asset_valid -m 'bleu,sari,fkgl' -q < %s" %pred_path).read()
  elif phase=='test':
    return os.popen("easse evaluate -t asset_test -m 'bleu,sari,fkgl' -q < %s" %pred_path).read()

def evaluate_my_model(output_id,generate_id,dataset,phase,output_file_name,ABCD=False,muss=False,simple_or_complex=None):#,use_token=True,use_simple=False):
  print('changed')
  if muss:
    result = ast.literal_eval(os.popen("easse evaluate -t asset_test -m 'bleu,sari,fkgl' -q < %s" %output_id).read())
    print(result)
    print('changed')
    # if ABCD:
    #   NE_retain_mean,NE_retain_std = get_hard_word_retain_portion(phase,output_id)
    # else:
    #   NE_retain_mean,NE_retain_std = get_NER_retain_portion(simple_or_complex,output_id)

    return result['bleu'], result['sari'], result['fkgl']#,NE_retain_mean,NE_retain_std
  else:
    result = ast.literal_eval(easse_my_model_eval(output_id,generate_id,dataset,phase,output_file_name))

    print(result)
    print('model id:',output_id)
    print('dataset:',dataset)

    pred_path = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/'+output_id+'/'+generate_id+'/'+output_file_name

    # if ABCD:
    #   NE_retain_mean,NE_retain_std = get_hard_word_retain_portion(phase,pred_path)
    # else:
    #   NE_retain_mean,NE_retain_std = get_NER_retain_portion(simple_or_complex,pred_path)

    return result['bleu'], result['sari'], result['fkgl']#,NE_retain_mean,NE_retain_std

In [86]:
# a = !easse evaluate -t turkcorpus_test -m 'bleu,sari,fkgl' -q < easse/easse/resources/data/system_outputs/turkcorpus/test/ACCESS

# def model_eval_easse(data_dir, pred_dir, ):


# %cd easse
# %cd ..
!easse evaluate -t asset_test -m 'bleu,sari,fkgl' -q < easse/resources/data/system_outputs/turkcorpus/test/ACCESS



/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/easse
{'bleu': 75.985, 'sari': 40.126, 'fkgl': 7.63, 'quality_estimation': {'Compression ratio': 0.941, 'Sentence splits': 1.187, 'Levenshtein similarity': 0.867, 'Exact copies': 0.042, 'Additions proportion': 0.156, 'Deletions proportion': 0.164, 'Lexical complexity score': 7.943}}


In [87]:
!easse evaluate -t turkcorpus_test -m 'bleu,sari,fkgl' -q < easse/resources/data/system_outputs/turkcorpus/test/ACCESS


{'bleu': 76.359, 'sari': 41.381, 'fkgl': 7.63, 'quality_estimation': {'Compression ratio': 0.941, 'Sentence splits': 1.187, 'Levenshtein similarity': 0.867, 'Exact copies': 0.042, 'Additions proportion': 0.156, 'Deletions proportion': 0.164, 'Lexical complexity score': 7.943}}


In [ ]:
from easse.sari import corpus_sari

corpus_sari(orig_sents=["About 95 species are currently accepted.", "The cat perched on the mat."],  
            sys_sents=["About 95 you now get in.", "Cat on mat."], 
            refs_sents=[["About 95 species are currently known.", "The cat sat on the mat."],
                        ["About 95 species are now accepted.", "The cat is on the mat."],  
                        ["95 species are now accepted.", "The cat sat."]])

In [ ]:
from easse.sari import corpus_sari

corpus_sari(orig_sents=["About 95 species are currently accepted.", "The cat perched on the mat."],  
            sys_sents=["About 95 you now get in.", "Cat on mat."])

In [ ]:
bleu_list, sari_list, fkgl_list,NE_retain_mean_list,NE_retain_std_list = [],[],[],[],[]

for i in range(10):
  dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/test.simple.'+str(i)
  bleu, sari, fkgl = evaluate_my_model(dir,0,0,0,0,muss=True,simple_or_complex = 'simple')
  bleu_list.append(bleu)
  sari_list.append(sari)
  fkgl_list.append(fkgl)

In [ ]:
# read_lines('/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset_ne/test.complex')
